In [116]:
import pandas as pd
import numpy as np
import plotly.express as px
import plotly.graph_objects as go

# Load the data
data = pd.read_csv("D:\DRIVE\SCIENCE_Projects\GlobalDeltas_Edward\Data\86Deltas_Pop2030_Landcover_modif_data_regions_Analysis.csv")

#Adjust the data for each year (1975, 2020, and 2030) to have a minimum population value of 1000 for both bubble size and color
years = [1975, 2020, 2030]
for year in years:
    data[f'population_{year}_adj'] = np.where(data[f'population_{year}'] < 1000, 1000, data[f'population_{year}'])
    data[f'log_population_{year}_adj'] = np.log(data[f'population_{year}_adj'])
    data[f'AdjustedSize_{year}_cbrt'] = data[f'population_{year}_adj'].apply(lambda x: x**(1/3))

# Start by plotting the 2030 data (so it will be at the bottom)
fig1 = px.scatter_geo(
    data,
    lat='Lat',
    lon='Lon',
    size='AdjustedSize_2030_cbrt',
    size_max = 25,
    color='log_population_2030_adj',
    color_continuous_scale=px.colors.sequential.Plasma,
    hover_name='Delta Name',
    hover_data={
        'DeltaArea': True,
        'population_1975': True,
        'population_2020': True,
        'population_2030': True,
        'ESRI_built': True,
        'ESRI_agri': True,
        'NaturalArea': True,
        'GHM_mean_modification': True,
        'NaturalFraction': True,
        'AdjustedSize_2030_cbrt': False,
        'Lat': False,
        'Lon': False,
        'log_population_2030_adj': False
    },
    projection='natural earth',
)

# 2020
fig2 = px.scatter_geo(
    data,
    lat='Lat',
    lon='Lon',
    size='AdjustedSize_2020_cbrt',
    size_max = 25/1.11,
    color='log_population_2020_adj',
    color_continuous_scale=px.colors.sequential.Plasma,
    hover_name='Delta Name',
    hover_data={
        'DeltaArea': True,
        'population_1975': True,
        'population_2020': True,
        'population_2030': True,
        'ESRI_built': True,
        'ESRI_agri': True,
        'NaturalArea': True,
        'GHM_mean_modification': True,
        'NaturalFraction': True,
        'AdjustedSize_2020_cbrt': False,
        'Lat': False,
        'Lon': False,
        'log_population_2020_adj': False
    },
    projection='natural earth',
)

# 1975
fig3= px.scatter_geo(
    data,
    lat='Lat',
    lon='Lon',
    size='AdjustedSize_1975_cbrt',
    size_max = 25/2.34,#scale with ratio of ganges population in 1975 vs 2030(max)
    color='log_population_1975_adj',
    color_continuous_scale=px.colors.sequential.Plasma,
    hover_name='Delta Name',
    hover_data={
        'DeltaArea': True,
        'population_1975': True,
        'population_2020': True,
        'population_2030': True,
        'ESRI_built': True,
        'ESRI_agri': True,
        'NaturalArea': True,
        'GHM_mean_modification': True,
        'NaturalFraction': True,
        'AdjustedSize_1975_cbrt': False,
        'Lat': False,
        'Lon': False,
        'log_population_1975_adj': False
    },
    projection='natural earth',
)

fig=px.scatter_geo(projection='natural earth')
fig.add_traces(fig1._data)
fig.add_traces(fig2._data)
fig.add_traces(fig3._data)

# Adjust the colorbar tick labels to show the population values in powers of 10
tickvals = [np.log(10**i) for i in range(0, 9)]
ticktext = ['10^{}'.format(i) for i in range(0, 9)]
fig.update_layout(
    coloraxis_colorbar=dict(tickvals=tickvals, ticktext=ticktext, title="Population")
)


# Define sizeref for 1975
#desired_maximum_marker_size_1975 = 100
#sizeref_1975_cbrt = 1. * max(data['AdjustedSize_1975_cbrt']) / (desired_maximum_marker_size_1975**2)

# Adjust the marker sizes using the 1975 sizeref for proportional scaling
#fig.update_traces(marker=dict(sizeref=1, symbol='circle', opacity=0.75)) #sizeref=sizeref_1975_cbrt
fig.update_geos(lataxis_range=[-60, 90])


# Add delta names as text to each marker
fig.add_trace(
    go.Scattergeo(
        lat=data['Lat'],
        lon=data['Lon'],
        text=data['Number'].astype(str),
        mode='text',
        showlegend=False,
        textposition='bottom right', 
        textfont=dict(size=14, color='black')
    )
)


# Save the final figure as an HTML file
file_path = "Bubble_map_pop1975_to_2030.html"
fig.write_html(file_path)
fig.show()